In [1]:
from sklearn.ensemble import GradientBoostingClassifier
import pickle
from sklearn.metrics import ndcg_score
import pandas as pd
import gc

In [2]:
# training set
x = pd.read_csv("data/dummy/train.csv")

# val set 20%
x_val = x.sample(n=int(0.1*len(x)))
x = x.drop(x_val.index)

## TODO: date_time
x.drop(['click_bool', 'booking_bool', 'date_time'], axis=1, inplace=True)
x_val.drop(['click_bool', 'booking_bool', 'date_time'], axis=1, inplace=True)

# sort both on search id
x.sort_values('srch_id', ignore_index=True)
x_val.sort_values('srch_id', ignore_index=True)

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,random_bool,agg_comp_rate,agg_comp_perc,agg_comp_inv,target
0,1,12,187,219,85728,2,3.0,1,1.79,0.0072,...,4,0,1,1,-1.00,1,0.25,11.0,-0.25,0
1,1,12,187,219,56880,4,4.0,1,2.83,0.1028,...,4,0,1,1,-1.00,1,0.00,0.0,0.00,0
2,4,5,219,219,110813,2,4.0,0,2.40,0.0000,...,1,0,1,1,233.87,1,0.00,0.0,0.00,0
3,4,5,219,219,127808,2,2.5,0,3.00,0.0457,...,1,0,1,1,237.73,1,0.00,0.0,0.00,0
4,6,14,100,100,104251,3,4.0,1,1.95,0.0000,...,2,0,1,0,652.84,0,0.00,0.0,0.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495829,332782,19,158,158,99998,3,1.0,0,0.69,0.0044,...,1,0,1,1,-1.00,0,0.00,0.0,0.00,0
495830,332782,19,158,158,9800,3,3.0,0,2.89,0.0493,...,1,0,1,1,-1.00,0,0.00,0.0,0.00,0
495831,332784,24,219,100,121275,4,4.0,0,4.98,0.0385,...,1,0,1,0,329.68,0,0.00,0.0,0.00,0
495832,332784,24,219,100,34966,3,4.5,0,5.04,0.0995,...,1,0,1,0,329.31,0,0.00,0.0,0.00,5


In [3]:
y_val = x_val.target.squeeze()
y = x.target.squeeze()
x.drop(['target'], axis=1, inplace=True)
x_val.drop(['target'], axis=1, inplace=True)

In [4]:
groups = x.srch_id.value_counts().sort_index()
groups_val = x_val.srch_id.value_counts().sort_index()

In [7]:
import lightgbm

# default lightgbm model with sklearn api
gbm = lightgbm.LGBMRanker(metric="ndcg", objective="lambdarank", num_boost_round=1000) 

#label_gain

##rank_xendcg, XE_NDCG_MART ranking objective function, aliases: xendcg, xe_ndcg, xe_ndcg_mart, xendcg_mart
#rank_xendcg is faster than and achieves the similar performance as lambdarank

# fitting model 
gbm.fit(
    x,
    y,
    eval_set = [(x_val, y_val)],
    group = groups,
    eval_group = [groups_val],
    verbose=1,
    early_stopping_rounds=100
)

/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's ndcg@1: 0.939944	valid_0's ndcg@2: 0.957953	valid_0's ndcg@3: 0.966419	valid_0's ndcg@4: 0.970156	valid_0's ndcg@5: 0.971651
Training until validation scores don't improve for 100 rounds
[2]	valid_0's ndcg@1: 0.941775	valid_0's ndcg@2: 0.959942	valid_0's ndcg@3: 0.967934	valid_0's ndcg@4: 0.971422	valid_0's ndcg@5: 0.972822
[3]	valid_0's ndcg@1: 0.943118	valid_0's ndcg@2: 0.960707	valid_0's ndcg@3: 0.968653	valid_0's ndcg@4: 0.972061	valid_0's ndcg@5: 0.973414
[4]	valid_0's ndcg@1: 0.943599	valid_0's ndcg@2: 0.961209	valid_0's ndcg@3: 0.968938	valid_0's ndcg@4: 0.972315	valid_0's ndcg@5: 0.973676
[5]	valid_0's ndcg@1: 0.943605	valid_0's ndcg@2: 0.961236	valid_0's ndcg@3: 0.968955	valid_0's ndcg@4: 0.972315	valid_0's ndcg@5: 0.97369
[6]	valid_0's ndcg@1: 0.943674	valid_0's ndcg@2: 0.96132	valid_0's ndcg@3: 0.969033	valid_0's ndcg@4: 0.972338	valid_0's ndcg@5: 0.973739
[7]	valid_0's ndcg@1: 0.94392	valid_0's ndcg@2: 0.961472	valid_0's ndcg@3: 0.969113	valid_0's ndcg@4: 0.97

[60]	valid_0's ndcg@1: 0.944368	valid_0's ndcg@2: 0.96191	valid_0's ndcg@3: 0.969457	valid_0's ndcg@4: 0.972781	valid_0's ndcg@5: 0.974058
[61]	valid_0's ndcg@1: 0.944333	valid_0's ndcg@2: 0.961878	valid_0's ndcg@3: 0.969454	valid_0's ndcg@4: 0.972771	valid_0's ndcg@5: 0.974037
[62]	valid_0's ndcg@1: 0.944373	valid_0's ndcg@2: 0.961909	valid_0's ndcg@3: 0.969429	valid_0's ndcg@4: 0.972776	valid_0's ndcg@5: 0.974049
[63]	valid_0's ndcg@1: 0.944453	valid_0's ndcg@2: 0.96193	valid_0's ndcg@3: 0.969486	valid_0's ndcg@4: 0.972805	valid_0's ndcg@5: 0.974081
[64]	valid_0's ndcg@1: 0.944419	valid_0's ndcg@2: 0.9619	valid_0's ndcg@3: 0.969463	valid_0's ndcg@4: 0.972798	valid_0's ndcg@5: 0.974066
[65]	valid_0's ndcg@1: 0.944339	valid_0's ndcg@2: 0.961884	valid_0's ndcg@3: 0.969437	valid_0's ndcg@4: 0.972761	valid_0's ndcg@5: 0.974038
[66]	valid_0's ndcg@1: 0.944379	valid_0's ndcg@2: 0.961931	valid_0's ndcg@3: 0.969448	valid_0's ndcg@4: 0.972777	valid_0's ndcg@5: 0.974064
[67]	valid_0's ndcg@1: 0

[119]	valid_0's ndcg@1: 0.944408	valid_0's ndcg@2: 0.961877	valid_0's ndcg@3: 0.969548	valid_0's ndcg@4: 0.972784	valid_0's ndcg@5: 0.97408
[120]	valid_0's ndcg@1: 0.944414	valid_0's ndcg@2: 0.961911	valid_0's ndcg@3: 0.969536	valid_0's ndcg@4: 0.972791	valid_0's ndcg@5: 0.974089
[121]	valid_0's ndcg@1: 0.944397	valid_0's ndcg@2: 0.961917	valid_0's ndcg@3: 0.969538	valid_0's ndcg@4: 0.972784	valid_0's ndcg@5: 0.974082
[122]	valid_0's ndcg@1: 0.944426	valid_0's ndcg@2: 0.961935	valid_0's ndcg@3: 0.969566	valid_0's ndcg@4: 0.972807	valid_0's ndcg@5: 0.974095
[123]	valid_0's ndcg@1: 0.944454	valid_0's ndcg@2: 0.961923	valid_0's ndcg@3: 0.969587	valid_0's ndcg@4: 0.972804	valid_0's ndcg@5: 0.974098
[124]	valid_0's ndcg@1: 0.944448	valid_0's ndcg@2: 0.961887	valid_0's ndcg@3: 0.969555	valid_0's ndcg@4: 0.972789	valid_0's ndcg@5: 0.974084
[125]	valid_0's ndcg@1: 0.94446	valid_0's ndcg@2: 0.961915	valid_0's ndcg@3: 0.969557	valid_0's ndcg@4: 0.972801	valid_0's ndcg@5: 0.974092
[126]	valid_0's

[179]	valid_0's ndcg@1: 0.94455	valid_0's ndcg@2: 0.96197	valid_0's ndcg@3: 0.969628	valid_0's ndcg@4: 0.972864	valid_0's ndcg@5: 0.974133
[180]	valid_0's ndcg@1: 0.944602	valid_0's ndcg@2: 0.961985	valid_0's ndcg@3: 0.969652	valid_0's ndcg@4: 0.972886	valid_0's ndcg@5: 0.974153
[181]	valid_0's ndcg@1: 0.944642	valid_0's ndcg@2: 0.962004	valid_0's ndcg@3: 0.969668	valid_0's ndcg@4: 0.972901	valid_0's ndcg@5: 0.974168
[182]	valid_0's ndcg@1: 0.944613	valid_0's ndcg@2: 0.961981	valid_0's ndcg@3: 0.969652	valid_0's ndcg@4: 0.972882	valid_0's ndcg@5: 0.974156
[183]	valid_0's ndcg@1: 0.944585	valid_0's ndcg@2: 0.961962	valid_0's ndcg@3: 0.969635	valid_0's ndcg@4: 0.972869	valid_0's ndcg@5: 0.974139
[184]	valid_0's ndcg@1: 0.944596	valid_0's ndcg@2: 0.96198	valid_0's ndcg@3: 0.969648	valid_0's ndcg@4: 0.972881	valid_0's ndcg@5: 0.974148
[185]	valid_0's ndcg@1: 0.944642	valid_0's ndcg@2: 0.96199	valid_0's ndcg@3: 0.96966	valid_0's ndcg@4: 0.972897	valid_0's ndcg@5: 0.974163
[186]	valid_0's nd

[240]	valid_0's ndcg@1: 0.944671	valid_0's ndcg@2: 0.962033	valid_0's ndcg@3: 0.969647	valid_0's ndcg@4: 0.972901	valid_0's ndcg@5: 0.974172
[241]	valid_0's ndcg@1: 0.944683	valid_0's ndcg@2: 0.962037	valid_0's ndcg@3: 0.969643	valid_0's ndcg@4: 0.972907	valid_0's ndcg@5: 0.974173
[242]	valid_0's ndcg@1: 0.944705	valid_0's ndcg@2: 0.962027	valid_0's ndcg@3: 0.969643	valid_0's ndcg@4: 0.972905	valid_0's ndcg@5: 0.974176
[243]	valid_0's ndcg@1: 0.944734	valid_0's ndcg@2: 0.962026	valid_0's ndcg@3: 0.969648	valid_0's ndcg@4: 0.972908	valid_0's ndcg@5: 0.974182
[244]	valid_0's ndcg@1: 0.944717	valid_0's ndcg@2: 0.962024	valid_0's ndcg@3: 0.969647	valid_0's ndcg@4: 0.972908	valid_0's ndcg@5: 0.974181
[245]	valid_0's ndcg@1: 0.944705	valid_0's ndcg@2: 0.96202	valid_0's ndcg@3: 0.969646	valid_0's ndcg@4: 0.972898	valid_0's ndcg@5: 0.974177
[246]	valid_0's ndcg@1: 0.944711	valid_0's ndcg@2: 0.962022	valid_0's ndcg@3: 0.96965	valid_0's ndcg@4: 0.972898	valid_0's ndcg@5: 0.974178
[247]	valid_0's

LGBMRanker(metric='ndcg', num_boost_round=1000, objective='lambdarank')

In [8]:
gc.collect()
with open('model/model', 'wb') as f:
    pickle.dump(gbm, f)

In [9]:
with open('model/model', 'rb') as f:
    model = pickle.load(f)

In [10]:
test = pd.read_csv("data/dummy/test.csv", index_col=False)
test.drop('date_time', axis=1, inplace=True)

In [11]:
predictions = model.predict(test)

In [12]:
len(predictions)

4959183

In [13]:
submission = test[["srch_id", "prop_id"]]
submission['pred'] = predictions

<ipython-input-13-f49d9f7448f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['pred'] = predictions


In [14]:
del test
gc.collect()

40

In [15]:
submission = submission.sort_values(['srch_id', 'pred'], ascending=[True, False])

In [16]:
submission.head(n=20)

,srch_id,prop_id,pred
23,1,99484,1.690570
9,1,54937,1.489587
12,1,61934,1.336383
4,1,24194,1.040477
5,1,28181,1.002976
8,1,50162,0.769798
6,1,34263,0.744481
20,1,90385,0.566053
18,1,82231,0.560829
13,1,63894,0.502426


In [17]:
submission[['srch_id', 'prop_id']].to_csv('submission.csv', index=False)